# The Refinery Files 0x07: DCRat

This is a short tutorial on how to reproduce the steps from [a recent blog post on DCRat][blog].
We will be working with the following sample:

[blog]: https://embee-research.ghost.io/dcrat-manual-de-obfuscation/

In [1]:
from tutorials import boilerplate as bp
bp.store_sample('fd687a05b13c4f87f139d043c4d9d936b73762d616204bfb090124fd163c316e', 'dc.rat')

## Unpacking Step 1

The blog post explains how to dump the first stage using dynamic analysis, but we'll do it statically.
The payload is obfuscated using a custom encoding routine, I suspect a dynamically generated one that is specific to ConfuserEx,
but I do not know for sure.
First, let's extract the encoded data itself - this can be done using the [dnfields] unit, which extracts the constant initialization values for fields in .NET assemblies.

[dnfields]: https://binref.github.io/#refinery.dnfields

In [1]:
%emit dc.rat | dnfields [| peek ]

------------------------------------------------------------------------------------------------------------------------
00.315 MB; 99.99% entropy; data
  name = DataField
  path = DataField.uint32
  type = uint32[78747]
------------------------------------------------------------------------------------------------------------------------
00000: 44 1F 17 70 B3 21 63 B9 9E 54 C4 DE 0E 66 11 52 8B 2F 14 0B 95 6B E0 2D 6B A8 23 11  D..p.!c..T...f.R./...k.-k.#.
0001C: B9 10 EB 7C 63 87 5A E6 BC 98 9F 2A 20 36 1B 38 B6 74 E9 A2 51 36 8C 3A 0E B3 58 E6  ...|c.Z....*.6.8.t..Q6.:..X.
00038: 39 8B 3B 34 38 5E 2E E2 DA 2D D4 0F CF 3E FC 0B A6 F7 06 F8 D4 F6 D7 BD EA EC A7 23  9.;48^...-...>.............#
00054: 5B C9 56 F5 C9 F1 4F 44 6A 3C 05 39 FB 83 7D D8 6C 97 37 88 D6 A6 D2 26 80 6D D7 C8  [.V...ODj<.9..}.l.7....&.m..
00070: 57 F8 84 24 7A A4 9B B0 A5 A2 C8 29 4F 47 5E 4F FC 30 18 89 D6 7E 57 3C EF 09 98 C7  W..$z......)OG^O.0...~W<....
0008C: 2D E8 29 10 90 1E 6C B7 25 FB 11 63 2E DA E4 E

There is only one, really large field - so at least we do not have to figure out which one to pick from among many options.
To reproduce the decryption, simply open the sample in [dnSpyEx], copy out the code from the function `Decrypt` (token `06000001`),
and convert it to a refinery unit:

[dnSpyEx]: https://github.com/dnSpyEx/dnSpy

In [1]:
#!binary-refinery
from refinery import Unit
from refinery import decompress

import array as ar

class decrypt(Unit):
    def process(self, data):
        array = [0] * 16
        array2 = [0] * 16
        num = 306067877
        for i in range(16):
            num = num * num % 339722377
            array2[i] = num
            array[i] = num * num % 1145919227
        array[0] = array[0] + array2[0] + 3017868035
        array[1] = (array[1] ^ array2[1]) + 3017868035
        array[2] = array[2] + array2[2] + 3017868035
        array[3] = array[3] * array2[3] * 2690889427
        array[4] = (array[4] * array2[4]) ^ 2765468969
        array[5] = (array[5] ^ array2[5]) * 2690889427
        array[6] = array[6] ^ array2[6] ^ 2765468969
        array[7] = (array[7] * array2[7]) ^ 2765468969
        array[8] = (array[8] ^ array2[8]) + 3017868035
        array[9] = (array[9] ^ array2[9]) + 3017868035
        array[10] = (array[10] ^ array2[10]) + 3017868035
        array[11] = (array[11] ^ array2[11]) + 3017868035
        array[12] = (array[12] ^ array2[12]) + 3017868035
        array[13] = (array[13] ^ array2[13]) + 3017868035
        array[14] = (array[14] ^ array2[14]) + 3017868035
        array[15] = (array[15] ^ array2[15]) + 3017868035
        array2 = [0] * 16
        array3 = ar.array('L', data)
        for j, c in enumerate(array3):
            num3 = c ^ array[j & 15]
            array[j & 15] = (array[j & 15] ^ num3) + 1037772825
            array3[j] = num3 & 0xFFFFFFFF
        array4 = array3.tobytes() | decompress | bytearray
        for k, b in enumerate(array4):
            array4[k] = b ^ (num & 0xFF)
            if k & 255 == 0:
                num = num * num % 9067703
        return array4

I wasn't sure what sort of decompression the .NET code uses, but luckily,
refinery already has a generic [decompress] unit which can usually determine and use the correct decompression method.
If you didn't know, you can use existing refinery units in Python code with syntax very similar to the command-line syntax:
```python
array4 = array3.tobytes() | decompress | bytearray
```
I do this because there is another decryption step following the decompression and I did not feel like splitting this part into several custom units.
Armed with this script, we can already decrypt the second stage with relative ease:

[decompress]: https://binref.github.io/#refinery.decompress

In [1]:
%emit dc.rat | dnfields | ./decrypt.py | dump -t koi.exe | peek -mm

------------------------------------------------------------------------------------------------------------------------
    crc32 = 861f07aa
  entropy = 43.74%
    magic = PE32 executable (GUI) Intel 80386 Mono/.Net assembly, for MS Windows
   sha256 = e62e3e03c6d5ce19267e343b2f22d4815ca1e6e6f714b1f36b1f3a4a45813a00
     size = 00.876 MB
------------------------------------------------------------------------------------------------------------------------
00000: 4D 5A 90 00 03 00 00 00 04 00 00 00 FF FF 00 00 B8 00 00 00 00 00 00 00 40 00 00 00  MZ......................@...
0001C: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ............................
00038: 00 00 00 00 80 00 00 00 0E 1F BA 0E 00 B4 09 CD 21 B8 01 4C CD 21 54 68 69 73 20 70  ................!..L.!This.p
00054: 72 6F 67 72 61 6D 20 63 61 6E 6E 6F 74 20 62 65 20 72 75 6E 20 69 6E 20 44 4F 53 20  rogram.cannot.be.run.in.DOS.
00070: 6D 6F 64 65 2E 0D 0D 0A 24 00 00 00 00 00 00 00

## Unpacking Step 2

Now the next step is to extract the final payload from `koi.exe`.
First, we use [dnstr] to extract all .NET user strings from the sample.
We then use [iff] to filter for only those strings that are longer than 200 characters.
The following command also uses [pick] to restrict the output to only 8 lines:

[dnstr]: https://binref.github.io/#refinery.dnstr
[pick]: https://binref.github.io/#refinery.pick
[iff]: https://binref.github.io/#refinery.iff
[transpose]: https://binref.github.io/#refinery.transpose

In [1]:
%emit koi.exe | dnstr -u [| iff size -ge 200 | pick :8 | peek -be ]

08.616 kB: TAT1IAAAAAAAAAAAAAAQAAAAAAAUAAAACsD4D8DcGsHwH0AAAAAQBEFcGoGAAAAEAAA4BDAAtMA8ACAATEAoACAAoIAwAAAA4MA8AAAAQIAcA
08.616 kB: VAMvgABAAACAAAAAQAFAAFAAQABAAIAAQLQOgPwXhbBcRAAAEAAAgRRXBaxAAKAAAABAQhAAAAFCAuAAQAHCAXwAAAIDAVAAQAOBAMwAQAMCA
08.616 kB: qA0ZEAAAAA5ABAUAAA0AAAAYMAAAAAA4kC0D4D8GoGwH0AAAAAIAcEUFgGsAAAA+AAQBAAATEA4ABAA9IAcAAAAaAAUADAATIAsACAAkEAMAB
08.616 kB: QAhGLgACAA0GAAAAAFAAFAAQAFAAAAAAJwNgOwPBaxcBdAAAAAAAIRVxYhbAFAAAAAAQDQAwANDApAAAAIBAHwAgAKCAmwAgACCAJQAAAGCAe
08.616 kB: AAVUAAAAAAZAAArAAEAAAAAAAoAAAAMACwD4D8DgGsHwHAIMAAAACEFcGoGAAAD0AIBAAAxEAEACAAvEAIACAAgIAYABAAgAAkACAA+IA8AAA
08.616 kB: AAGuQABFAAXuAAAAFAAFAARAFAALAAAAQMQOgPwYhbBcRAAAAAAEARRXBaxBAAQEAAQeAAgABBAWAAQAABAlQAAAEDAhgAAAFDAPgAAAEDApg
08.616 kB: MAhDgBAAAAhcBQUAcAAAAAgAQAAAACAAkD0D4D8GoGwH0AAABAAAkEUFkGsAACAAAAAAUoEAYACAAhEAEACAAvEAIABAAfAAIACAAnIAAACAA
08.616 kB: AApQAAABAA0nAADAAAFAAgAFAARAAgAAJANgOwPBaxcBdQAAgACAJhVxZhbAAgAOAAMAAwAMBAvAAwAFCAqwAgABCAfgAQAGCACgAgAGCAXQA


We can already see the familiar `TVqQAA` pattern of a base64-encoded PE header when reading the output column-wise.
The unit [transpose] can be used to perform the operation that we are looking for here, and subsequently we use [b64] to decode the result.

[b64]: https://binref.github.io/#refinery.b64
[pad]: https://binref.github.io/#refinery.pad
[transpose]: https://binref.github.io/#refinery.transpose

In [1]:
%emit koi.exe | dnstr -u [| iff size -ge 200 | transpose ]| b64 | dump -t payload.bin | peek -mm

------------------------------------------------------------------------------------------------------------------------
    crc32 = a07f4088
  entropy = 72.00%
    magic = PE32 executable (GUI) Intel 80386 Mono/.Net assembly, for MS Windows
   sha256 = f6b193ae794a423a4cd5a4dcd284437823336658d1d0752b48c297a02d5fb46a
     size = 00.323 MB
------------------------------------------------------------------------------------------------------------------------
00000: 4D 5A 90 00 03 00 00 00 04 00 00 00 FF FF 00 00 B8 00 00 00 00 00 00 00 40 00 00 00  MZ......................@...
0001C: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ............................
00038: 00 00 00 00 80 00 00 00 0E 1F BA 0E 00 B4 09 CD 21 B8 01 4C CD 21 54 68 69 73 20 70  ................!..L.!This.p
00054: 72 6F 67 72 61 6D 20 63 61 6E 6E 6F 74 20 62 65 20 72 75 6E 20 69 6E 20 44 4F 53 20  rogram.cannot.be.run.in.DOS.
00070: 6D 6F 64 65 2E 0D 0D 0A 24 00 00 00 00 00 00 00

## Config Extraction

Now that we have the payload extracted, let's extract the config data:

In [1]:
%%emit payload.bin | carve -lt4 b64 [
    | b64
    | zl
    | rev
    | scope 1
    | b64
    | xtjson SCRT
    | xtjson
    | cfmt {path}{}
    | transpose
    | pop d s
    | max size
    | map eat:s eat:d
    | b64
    | xtjson H1 H2
    | defang ]]

http[:]//battletw.beget[.]tech/@==wYpxmY1BHdzVWdxVmc
http[:]//battletw.beget[.]tech/@==wYpxmY1BHdzVWdxVmc
